In [1]:
from engibench.problems.power_electronics.source import power_electronics_v0

# Create a problem
original_netlist_path = "engibench/problems/power_electronics/data/netlist/5_4_3_6_10-dcdc_converter_1.net"  # sweep 141
bucket_id = "5_4_3_6_10"
problem = power_electronics_v0.build(original_netlist_path=original_netlist_path, bucket_id=bucket_id)

d:\git_repos\EngiBench\engibench\problems\power_electronics\source
d:\git_repos\EngiBench\engibench\problems\power_electronics\ngSpice64\bin\ngspice.exe


In [2]:
sweep_data = {
    "C_val": [0.000015485, 0.00001948, 0.000015185, 0.000002442, 0.000009287, 0.000015377],
    "L_val": [0.000354659, 0.000706596, 0.000195361],
    "T1_val": [0.867615857, 0.867615857, 0.867615857, 0.867615857, 0.867615857],
    "T2_val": [1, 1, 1, 1, 1],
    "L1_val": [1, 1, 0, 0, 1],
    "L2_val": [1, 1, 1, 0, 0],
}
problem.simulate(design_variable=sweep_data)
print(problem.simulation_results)

Processing topology from original netlist path: engibench/problems/power_electronics/data/netlist/5_4_3_6_10-dcdc_converter_1.net
rewriting netlist to: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\rewrite_control_5_4_3_6_10-dcdc_converter_1.net
Running command: ['d:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\ngSpice64\\bin\\ngspice.exe', '-o', '../data/log_file/5_4_3_6_10-dcdc_converter_1.log', 'd:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\data\\netlist\\rewrite_control_5_4_3_6_10-dcdc_converter_1.net']
rawread.py: mdata: [b'Title', b' * rewrite netlist\n']
mdata len = 2 [b'Title', b' * rewrite netlist\n']
rawread.py: mdata: [b'Date', b' Fri Dec 13 04:10:04  2024\n']
mdata len = 2 [b'Date', b' Fri Dec 13 04:10:04  2024\n']
rawread.py: mdata: [b'Plotname', b' Transient Analysis\n']
mdata len = 2 [b'Plotname', b' Transient Analysis\n']
rawread.py: mdata: [b'Flags', b' real\n']
mdata len = 2 [b'Flags', b' real\n']
rawrea